In [93]:
# import libraries
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import os
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from tqdm import tqdm

In [94]:
#########사용법#########
"""
기본적인 code들은 영서님과 제가 업로드한 code를 이용해서 구현했습니다.
우선, 사용할 모델과 학습 데이터셋을 결정하시고 나서 해당 code를 수정해서 돌리면 됩니다.
해당 code는 ./Frame./test_codes./모델_데이터셋.ipynb 형태로 저장해서 돌리시면 됩니다.

해당 모델은 xgboost로 월~금의 데이터를 기반으로 다음주 월, 화~월의 데이터로 화요일의 종가를 예측합니다.
learning rate를 적절히 조정하면 4.345정도의 결과값을 보입니다.
"""
########################

'\n기본적인 code들은 영서님과 제가 업로드한 code를 이용해서 구현했습니다.\n우선, 사용할 모델과 학습 데이터셋을 결정하시고 나서 해당 code를 수정해서 돌리면 됩니다.\n해당 code는 ./Frame./test_codes./모델_데이터셋.ipynb 형태로 저장해서 돌리시면 됩니다.\n\n해당 모델은 xgboost로 월~금의 데이터를 기반으로 다음주 월, 화~월의 데이터로 화요일의 종가를 예측합니다.\nlearning rate를 적절히 조정하면 4.345정도의 결과값을 보입니다.\n'

In [95]:
# 기본적으로 사용하는 함수들
# columns rename function
def cols_rename(data_set, target_name):
    for i in data_set.columns:
        if i == 'Date':
            pass
        else:
            data_set.rename(columns={i:target_name+'_'+i}, inplace=True)
    return data_set
# columns rename function which has integer column name
def cols_int_rename(data_set, target_name):
    for i in data_set.columns:
        if i == 'Date':
            pass
        else:
            data_set.rename(columns={i:target_name+'_'+str(i)}, inplace=True)
    return data_set
# column rename function
def col_rename(data_set, name_from, name_to):
    for i in data_set.columns:
        if i != name_from:
            pass
        else:
            data_set.rename(columns={i:name_to}, inplace=True)
    return data_set
# function for adjusting friday to sunday data as new friday data
def weekendToFriday(target, idx, col, returnType): # idx: Friday
    if (returnType == 'intType'):
        target.loc[idx, col] = (target.loc[idx, col]//7)*1 + (target.loc[idx+1, col]//7)*2 + (target.loc[idx+2, col]//7)*4
    elif (returnType == 'floatType'):
        target.loc[idx, col] = (target.loc[idx, col]/7)*1 + (target.loc[idx+1, col]/7)*2 + (target.loc[idx+2, col]/7)*4
    return target.loc[idx, col]

In [96]:
#세팅해야 하는 변수들

# 데이터 수집 기간 설정
#start_date = '20200106'
start_date = '20140106'
#start_date = '20201019'
end_date = '20220520'

# 기본적인 data를 불러오는 folder path
path = '../data'
list_name = 'stock_list.csv'
sample_name = 'sample_submission.csv'

stock_list = pd.read_csv(os.path.join(path, list_name))

# 호용님 code에 유사하게 사용
stock_list = stock_list.sort_values(by=['종목코드'])
# 없는 것들은 삭제: 031390 ,036490
stock_list.drop(stock_list[stock_list['종목코드']==31390].index,inplace=True)
stock_list.drop(stock_list[stock_list['종목코드']==36490].index,inplace=True)
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))

In [97]:
######################################
# 학습시킬 data를 선정하는 code 입니다
######################################

# make business_days: 주말 미포함
# make business_days: 주말 미포함
Business_days = pd.DataFrame(pd.date_range(start_date, end_date, freq='B'), columns = ['Date'])
Business_days['weekday'] = Business_days.Date.apply(lambda x: x.weekday())
Business_days['weeknum'] = Business_days.Date.apply(lambda x: x.strftime('%V'))


################################################################################
# 이 사이에다가 추가하고 싶으신 데이터를 넣으시면 됩니다.
"""
# 기존에 사용하던 데이터 - 우선 남겨두고, 영서님께서 해주신 code를 이용해서 했습니다.
# KOSPI : 최대한 건들지 않는게 좋을 것 같습니다.
KOSPI = fdr.DataReader('KS11', start_date, end_date).reset_index()
KOSPI = KOSPI.drop(['Open', 'High', 'Low'], axis=1)
cols_rename(KOSPI, 'kospi')
data = pd.merge(Business_days, KOSPI, how='outer')

# KOSDAQ 추가
KOSDAQ = fdr.DataReader('KQ11', start_date, end_date).reset_index()
KOSDAQ = KOSDAQ.drop(['Open', 'High', 'Low'], axis=1)
cols_rename(KOSDAQ, 'kosdaq')
data = pd.merge(data, KOSDAQ, how='outer')

# 미국증시: 나스닥(NASDAQ) 추가
NAS = fdr.DataReader('NASDAQCOM', start_date, end_date, data_source='fred').reset_index()
col_rename(NAS, 'DATE', 'Date')
col_rename(NAS, 'NASDAQCOM', 'nasdaq_Close')
data = pd.merge(data, NAS, how='outer')

# 환율: 원달러(USD) 추가
USD = fdr.DataReader('USD/KRW', start_date, end_date).reset_index()
USD = USD.drop(['Open', 'High', 'Low'], axis=1)
cols_rename(USD, 'usd')
data = pd.merge(data, USD, how='outer')

# 환율: 원엔(JPY) 추가
JPY = fdr.DataReader('JPY/KRW', start_date, end_date).reset_index()
JPY = JPY.drop(['Open', 'High', 'Low'], axis=1)
cols_rename(JPY, 'jpy')
data = pd.merge(data, JPY, how='outer')

# 환율: 호주달러/스위스프랑 추가
ACF = fdr.DataReader('AUD/CHF', start_date, end_date).reset_index()
ACF = ACF.drop(['Open', 'High', 'Low'], axis=1)
cols_rename(ACF, 'acf')
data = pd.merge(data, ACF, how='outer')

#10년 단위 미 국채
#US_10 = fdr.DataReader('US10YT=X', start_date, end_date).reset_index()
#US_10 = US_10.drop(['Open', 'High', 'Low'], axis=1)
# 일~금 데이터만 있고 토요일 데이터는 없기 때문에 해당 부분 반영
#cols_rename(US_10, 'us_10')
#data = pd.merge(data, US_10, how='outer')

# 변동성모델(CBOE에서 제공하는)
VIX = fdr.DataReader('VIX', start_date, end_date).reset_index()
VIX = VIX.drop(['Open', 'High', 'Low'], axis=1)
cols_rename(VIX, 'vix')
data = pd.merge(data, VIX, how='outer')

"""
# 주중 데이터
# KOSPI 추가
KOSPI = fdr.DataReader('KS11', start_date, end_date).reset_index()
KOSPI.drop(['Open', 'High', 'Low'], axis=1, inplace=True)
cols_rename(KOSPI, 'kospi')
data = pd.merge(Business_days, KOSPI, how='outer')
# KOSDAQ 추가
KOSDAQ = fdr.DataReader('KQ11', start_date, end_date).reset_index()
KOSDAQ.drop(['Open', 'High', 'Low'], axis=1, inplace=True)
cols_rename(KOSDAQ, 'kosdaq')
data = pd.merge(data, KOSDAQ, how='outer')
# 미국증시: 나스닥(NASDAQ) 추가
NAS = fdr.DataReader('NASDAQCOM', start_date, end_date, data_source='fred').reset_index()
col_rename(NAS, 'DATE', 'Date')
col_rename(NAS, 'NASDAQCOM', 'nasdaq_Close')
data = pd.merge(data, NAS, how='outer')
# 환율: 원달러(USD) 추가
USD = fdr.DataReader('USD/KRW', start_date, end_date).reset_index()
USD.drop(['Open', 'High', 'Low'], axis=1, inplace=True)
cols_rename(USD, 'usd')
data = pd.merge(data, USD, how='outer')
# 환율: 원엔(JPY) 추가
JPY = fdr.DataReader('JPY/KRW', start_date, end_date).reset_index()
JPY.drop(['Open', 'High', 'Low'], axis=1, inplace=True)
cols_rename(JPY, 'jpy')
data = pd.merge(data, JPY, how='outer')
# 환율: 호주달러/스위스프랑 추가
ACF = fdr.DataReader('AUD/CHF', start_date, end_date).reset_index()
ACF.drop(['Open', 'High', 'Low'], axis=1, inplace=True)
cols_rename(ACF, 'acf')
data = pd.merge(data, ACF, how='outer')
# 국채: 미 국채 10년 추가
UGB = fdr.DataReader('DGS10', start_date, end_date, data_source='fred').reset_index()
col_rename(UGB, 'DATE', 'Date')
col_rename(UGB, 'DGS10', 'ugb_Close')
data = pd.merge(data, UGB, how='outer')
## 변동성 지수 추가
VIX = fdr.DataReader('VIXCLS', start_date, end_date, data_source='fred').reset_index()
col_rename(VIX, 'DATE', 'Date')
col_rename(VIX, 'VIXCLS', 'vix_Close')
data = pd.merge(data, VIX, how='outer')


##############################
# 주말 데이터
# 암호화폐: 비트코인(BitCoin) 추가

BTC = fdr.DataReader('BTC/KRW',start_date,end_date).reset_index()
BTC['dayofweek'] = BTC['Date'].dt.dayofweek # 요일 (월 = 0)
BTC.drop(['Open', 'High', 'Low'], axis=1, inplace=True)
cols_rename(BTC, 'btc')

# 예외처리: 마지막 데이터가 토요일인 경우
if (BTC.loc[len(BTC)-1, 'btc_dayofweek'] == 5):
    BTC.drop(len(BTC)-1, inplace=True)

# 금토일 데이터를 금요일 데이터로 병합 후 주말 데이터 제거
# 금토일 데이터 가중치 >> 1:2:4
for idx in BTC.index:
    if (BTC.loc[idx, 'btc_dayofweek'] == 6):
        BTC.loc[idx-2, 'btc_Close'] = weekendToFriday(BTC, idx-2, 'btc_Close', 'intType')
        BTC.loc[idx-2, 'btc_Volume'] = weekendToFriday(BTC, idx-2, 'btc_Volume', 'floatType')
        BTC.loc[idx-2, 'btc_Change'] = np.ceil(weekendToFriday(BTC, idx-2, 'btc_Change', 'floatType')*1000)/1000
for idx in BTC.index:
    if (BTC.loc[idx, 'btc_dayofweek'] == 5 or BTC.loc[idx, 'btc_dayofweek'] == 6):
        BTC.drop(idx, inplace=True)

# 데이터 가공
BTC.drop(['btc_dayofweek'], axis=1, inplace=True)
BTC.reset_index(inplace=True)
BTC.drop(['index'], axis=1, inplace=True)

data = pd.merge(data, BTC, how='outer')


# 국채: 한국 국채 추가
KGB = fdr.DataReader('KR10YT=RR', start_date, end_date).reset_index()
KGB['dayofweek'] = KGB['Date'].dt.dayofweek # 요일 (월 = 0)
KGB.drop(['Open', 'High', 'Low'], axis=1, inplace=True)
cols_rename(KGB, 'kgb')

# 예외처리: 마지막 데이터가 토요일인 경우
if (KGB.loc[len(KGB)-1, 'kgb_dayofweek'] == 5):
    KGB.drop(len(KGB)-1, inplace=True)

# 금토일 데이터를 금요일 데이터로 병합 후 주말 데이터 제거
# 금토일 데이터 가중치 >> 1:2:4
for idx in KGB.index:
    if (KGB.loc[idx, 'kgb_dayofweek'] == 6):
        KGB.loc[idx-2, 'kgb_Close'] = weekendToFriday(KGB, idx-2, 'kgb_Close', 'floatType')
        KGB.loc[idx-2, 'kgb_Change'] = weekendToFriday(KGB, idx-2, 'kgb_Change', 'floatType')
        KGB.loc[idx-2, 'kgb_Change'] = np.ceil(weekendToFriday(KGB, idx-2, 'kgb_Change', 'floatType')*1000)/1000
for idx in KGB.index:
    if (KGB.loc[idx, 'kgb_dayofweek'] == 5 or KGB.loc[idx, 'kgb_dayofweek'] == 6):
        KGB.drop(idx, inplace=True)

# 데이터 가공
KGB.drop(['kgb_dayofweek'], axis=1, inplace=True)
KGB.reset_index(inplace=True)
KGB.drop(['index'], axis=1, inplace=True)

data = pd.merge(data, KGB, how='outer')

# NaN 값은 전일 값으로 대체. 
data = data.fillna(method='ffill')
data = data.fillna(method='bfill') # 예외처리



start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])




################################################################################



In [98]:
##############
# 휴장일 데이터에 대해서 처리해야한다.
# 휴장일 data는 삭제한다.
#kor_NAN_list = data[data['kospi_Close'].isnull()].index
#data.drop(set(kor_NAN_list.to_list()), inplace = True)
# 휴장일 데이터는 앞의 데이터를 가져온다
#print(data.isnull().sum())
kor_NAN_list = data[data['kospi_Close'].isnull()].index
#en_NAN_list = data[data['vix_Close'].isnull()].index
nan_list = data[data.isnull()]
#print(kor_NAN_list)
#print(nan_list)


# learn에 사용할 데이터
#train_data = data.drop(['weekday', 'weeknum'], axis=1)

In [99]:
data = data.fillna(method='ffill')
data = data.fillna(method='bfill')
#nan_list = set(kor_NAN_list.to_list() + en_NAN_list.to_list())
#data.drop(nan_list, inplace=True)
data[data['kospi_Close'].isnull()].index

Int64Index([], dtype='int64')

In [100]:
##############################################
# 특정 model에 대해서, 각 종목별로 학습하는 code 입니다
##############################################
import warnings
warnings.filterwarnings('ignore')

sample_name = 'sample_submission.csv'
sample_submission = pd.read_csv(os.path.join(path, sample_name))
true_close = sample_submission.copy()
true_close['Index'] = true_close['Index'].apply(lambda x : str(x).zfill(6))


day_list = ['2022-05-23', '2022-05-24', '2022-05-25', '2022-05-26', '2022-05-27']


i = 0
for code in tqdm(stock_list['종목코드'].values):
    # 상장폐지된 데이터는 0으로 해야하기 때문에 해당 부분을 더해준다.
    while code != true_close.iloc[i]['Index']:
        # print(f"{code}, {true_close.iloc[i]['Index']}")
        i += 1
    true_close.iloc[i,1:] = fdr.DataReader(code, start=day_list[0], end = day_list[-1])['Close']
    i += 1

true_close

100%|████████████████████████████████████████████████████████████████████████████████| 368/368 [00:26<00:00, 14.07it/s]


,Index,2022-05-23,2022-05-24,2022-05-25,2022-05-26,2022-05-27
0,000060,39750,39700,40800,39800,39800
1,000080,35350,34950,35900,36150,36250
2,000100,59600,59100,59700,59900,59800
3,000120,123000,125500,125500,127500,129000
4,000150,80400,79900,80400,80100,80400
...,...,...,...,...,...,...
365,348210,68400,66500,67000,66300,67000
366,352820,221000,215500,219500,214500,218500
367,357780,269900,260600,260600,270900,271800
368,363280,24500,24400,24550,24600,24400


In [101]:
##############################################
# 특정 model에 대해서, 각 종목별로 학습하는 code 입니다
##############################################
import warnings
warnings.filterwarnings('ignore')
submission = sample_submission.copy()
submission['Index'] = submission['Index'].apply(lambda x : str(x).zfill(6))
day_first = day_list[0]

In [102]:
submission

,Index,2022-05-23,2022-05-24,2022-05-25,2022-05-26,2022-05-27
0,000060,0,0,0,0,0
1,000080,0,0,0,0,0
2,000100,0,0,0,0,0
3,000120,0,0,0,0,0
4,000150,0,0,0,0,0
...,...,...,...,...,...,...
365,348210,0,0,0,0,0
366,352820,0,0,0,0,0
367,357780,0,0,0,0,0
368,363280,0,0,0,0,0


In [103]:
# Normalize the data

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scale_col = data.columns.to_list()
data_scaled = scaler.fit_transform(data[scale_col[3:]])
data_scaled = pd.DataFrame(data_scaled)
#print(data[scale_col[:3]])
data_scaled = pd.merge(data[scale_col[:3]], data_scaled, how='left', left_index=True, right_index=True)
data_scaled.columns = scale_col

In [104]:
# 종목별로 사용할 데이터 작성

ind_code = 0

# 데이터 정리
indasdf = 0
for code in tqdm(stock_list['종목코드'].values):
    #print(code)
    
    # x에 대해서 작성
    # 1주전의 데이터(월요일이라면, 저번주 월~금, 화요일이라면, 저번주 화~월)를 사용하고, 
    # x와 y index가 같더라도 y[x_index+1]을 답으로 삼기 때문에, 1, 2, 3, 4일 전 데이터를 사용하면 된다.
    
    # ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Change']
    code_data = fdr.DataReader(code, start = start_date, end = end_date).reset_index()   
    #code_data = code_data.drop(['Open', 'High', 'Low'], axis=1)
    code_data = pd.merge(Business_days, code_data, how='outer')
    #code_data = code_data[code_data['Close'].notna()]
    
    # NAN이 아닌 첫 index를 찾는다.
    
    #이동 평균선 추가: 5, 10, 20, 60일(1주 전)
    #code_data['5_close'] = code_data['Close'].rolling(window=5).mean()   
    #code_data['10_close'] = code_data['Close'].rolling(window=10).mean()   
    #code_data['20_close'] = code_data['Close'].rolling(window=20).mean()
    #code_data['60_close'] = code_data['Close'].rolling(window=60).mean() 
    
    
    #df = pd.merge(data, code_data, how='left', on='Date')
    df = pd.merge(data_scaled, code_data, how='left', on='Date')
    
    #df['weekday'] = df.Date.apply(lambda x : x.weekday())
    #df['weeknum'] = df.Date.apply(lambda x : x.strftime('%V'))
    for i in range(1, 5):
        df[f'Close_{i}d_bef'] = df['Close'].shift(i)    
    
    #print(pd.isna(df.to_numpy()).sum())
    df = df.ffill()
    #df = df.bfill()
    
    # 만약 KOSPI면, kosdaq 관련 내용 drop
    # 만약 이걸 drop하지 않으면 결과가 어떨까? 

    market = stock_list[stock_list['종목코드'].str.contains(code)]['상장시장'].values[0]
    if market == 'KOSPI':
        df = df.drop(['kosdaq_Close', 'kosdaq_Volume', 'kosdaq_Change'], axis=1)
    else:
        df = df.drop(['kospi_Close', 'kospi_Volume', 'kospi_Change'], axis=1)

    # code_data = code_data[code_data['Close'].notna()]
    # Business day의 시작 날짜보다, 상장한 날짜가 더 느린 경우를 생각하기 위해, close 기준으로 앞 부분 다 빼주기    
    df = df[df['Close'].notna()]
    
    #print(df.iloc[5:].isna().sum())
    # 날짜, 몇번째 주인지는 의미 없을듯 하고, 요일정도는 의미가 있지 않을까? weekday도 뺴자
    df = df.drop(['Date','weekday'], axis=1)
    #df = df.drop(['Date','weekday'], axis=1)
    #df = df.drop(['Date'], axis=1)

    x_data = df.to_numpy()
    #print(pd.isna(x_data[5:]).sum())
    #print(x_data.shape)

    # 이제 학습을 진행한다
    #print(df.columns.to_list())
    #break
    
    xgb_dict = dict()
    
    for name in df.columns.to_list():
        xgb_dict[name] = XGBRegressor(eta = 0.10, subsample=0.8)
        
        
    # y값의 [-1]: 5일전, [-2]: 4일전, [-3]: 3일전, [-2]: 2일전, ['Close']: 하루전->예측된다
    # 다만 돌리다보면 어느정도 확정되는 값이 존재하기 때문에, 해당 부분은 어느정도 감안해야한다.
    # ['kospi_Close', 'kospi_Volume', 'kospi_Change', 'nasdaq_Close','usd_Close',
    #  'usd_Change', 'jpy_Close', 'jpy_Change', 'acf_Close','acf_Change',
    #  'ugb_Close', 'vix_Close', 'Open', 'High', 'Low',
    #  'Close','Volume', 'Change', 'Close_2d_bef', 'Close_3d_bef',
    #  'Close_4d_bef','Close_5d_bef'],
    
    
    
    
    ind_close = df.columns.to_list().index('Close')
    #ind_weekday = df.columns.to_list().index('weekday')
    #ind_weeknum = df.columns.to_list().index('weeknum')
    predict_Close = []
    
    # 금요일 종가에 대한 비율을 계산해서 예측해보자.
    

    # 첫째날(월요일)
    ind_ = 0
    list_ = []
    for col_type in df.columns.to_list():
        xgb_dict[col_type].fit(x_data[5:-1], x_data[6:, ind_])
        ind_ += 1
        predict = xgb_dict[col_type].predict(np.array([x_data[-1]]))
        list_.append(predict[0])
    list_ = np.array(list_)
    predict_Close.append(list_[ind_close])
   

    # 둘째날
    list_input = list_
    list_output = []
    # 예측한 것 중에서, 2~5일정도 예측했기 때문에, 실제 데이터를 넣어준다.
    list_input[-1] = x_data[-1][-2]
    list_input[-2] = x_data[-1][-3]
    list_input[-3] = x_data[-1][-4]
    list_input[-4] = x_data[-1][ind_close]
    # weekday
    #list_input[ind_weekday] = 0 # 월요일
    # weeknum
    #list_input[ind_weeknum] = str(int(x_data[-1][ind_weeknum])+1)
            
    # 이제 예측한다
    for col_type in df.columns.to_list():
        predict = xgb_dict[col_type].predict([list_input])
        list_output.append(predict[0])
    list_output = np.array(list_output)
    predict_Close.append(list_output[ind_close])
    
    for i in range(3):
        # 셋째, 넷째, 다섯째날
        list_output[-1] = list_input[-2]
        list_output[-2] = list_input[-3]
        list_output[-3] = list_input[-4]
        list_output[-4] = list_input[ind_close]
        list_input = list_output
        list_output = []
        # 이제 예측한다
        for col_type in df.columns.to_list():
            predict = xgb_dict[col_type].predict([list_input])
            list_output.append(predict[0])
        list_output = np.array(list_output)
        predict_Close.append(list_output[ind_close])
    
    ind_ = 0
    predict_Close = np.asarray(predict_Close, dtype = int)
    
    while submission.iloc[ind_code]['Index'] != code:
        ind_code += 1
        
    submission.iloc[ind_code, 1:] = predict_Close


100%|████████████████████████████████████████████████████████████████████████████████| 368/368 [25:18<00:00,  4.13s/it]


In [105]:
#np.mean(np.mean(np.abs(submission[5:] - true_close[5:]) / true_close[5:]))*100: 잘못된 수식입니다. 호용님 code대로 했습니다.
# 호용님 code: print(diff / np.sum(list(test_y.values())))
submission_result = submission[submission.columns[1:]].to_numpy()
true_close_result = true_close[true_close.columns[1:]].to_numpy()

print(np.sum(np.sum(np.abs(submission_result- true_close_result)))/np.sum(np.sum(true_close_result))*100)
print(np.mean(np.abs(submission_result - true_close_result)))

2.6398563060689253
1942.6794594594594


In [17]:
submission.to_csv('submission.csv', index=False)